# AIO Computation function definitions

In [134]:
# Libraries
import cobra
import numpy as np
import pyEQL.chemical_formula

# function definitions

def GetComposition_old(formula,atom="C"):
    ch=""
    flag=False
    for c in formula:
        if c == atom:
            flag=True
        else:
            if c.isnumeric() and flag:
                ch=ch+c
            else:
                flag=False
    if ch == "":
        return 0
    else:
        return int(ch)


def GetComposition(formula,atom="C"):
#    print("formula : ",formula)
    if (formula == None or formula == ""):
        formula = "S"
    try:
        return pyEQL.chemical_formula.get_element_mole_ratio(formula,atom)
    except ValueError:
        return 0
    
def getSign(r,fva=0):
    if fva == 0:
        minim=r.lower_bound
        maxim=r.upper_bound
    else:
        minim=fva["minimum"].get(r.id)
        maxim=fva["maximum"].get(r.id)
    if (abs(minim)+abs(maxim)>0.01):
        if (minim*maxim<0):
            return 0
        else:
            if (maxim<=0):
                return -1
            else:
                return 1
    else:
        return 0
            

ReactInputTable=dict()
ReactProductTable=dict()

def ReactionInputs(r,sign=1):
    if not ReactInputTable.get(r.id):
        ReactInputTable[r.id] = ReactionInputsLong(r,sign)
    return ReactInputTable.get(r.id)
 
        
def ReactionProducts(r,sign=1):
    if not ReactProductTable.get(r.id):
        ReactProductTable[r.id] = ReactionProductsLong(r,sign)
    return ReactProductTable.get(r.id)
    
def ReactionInputsLong(r,sign=1):
    res=list()
    for m in r.metabolites:
        if (sign*r.get_coefficient(m)<-0.01):
            res.append(m)
    return res
        
def ReactionProductsLong(r,sign=1):
    res=list()
    for m in r.metabolites:
        if (sign*r.get_coefficient(m)>0.01):
            res.append(m)
    return res
    

def In(r,m,atom="C"):
    res=0
    if m in ReactionProducts(r,getSign(r)):
        res=abs(r.get_coefficient(m))*GetComposition(m.formula,atom)
        somme=0
        for met in ReactionInputs(r,getSign(r)):
            somme+=abs(r.get_coefficient(met))*GetComposition(met.formula,atom)
        if not somme == 0:
            res=res/somme
        else:
            res=1
    return res


def RescaleModel(model,frac=0.5):
    fva=cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum=frac).to_dict()
    for r in model.reactions:
        r.lower_bound = fva["minimum"].get(r.id)
        r.upper_bound = fva["maximum"].get(r.id)
        
        
    
def value_of_D(mk,mi,V,atom="C"):
    # case 1 : r is the exchange reactions for m input
    if mk == mi:
        return 1
    # General case
    res=0
    for r in isConsumedBy[mi.id]:
            mr=metRate(mi,V)
            if not mr==0: 
                res -= abs(r.get_coefficient(mi)) * V.get(r.id) *In(r,mk,atom) / mr
    return res 



def getMetabolitLists(model):
    LMet=list()
    for m in model.metabolites:
        LMet.append(m)
    LMetInputs=list()
    for m in LMet:
        flag=False
        for r in m.reactions:
            if len(ReactionInputs(r,getSign(r)))==0 and m in ReactionProducts(r,getSign(r)):
                flag=True
        if flag:
#            print(m.id,"is an input")
            LMetInputs.append(m)       
    LMetOutputs=list()
    for m in LMet:
        flag=False
        for r in m.reactions:
            if len(ReactionProducts(r,getSign(r)))==0 and m in ReactionInputs(r,getSign(r)):
                flag=True
        if flag:
#            print(m.id,"is an output")
            LMetOutputs.append(m)

    LMetOthers=list()
    for m in LMet:
        # Beurkkkk !!!!
        flag=True
        for r in m.reactions:
            if len(ReactionProducts(r,getSign(r)))==0 and m in ReactionInputs(r,getSign(r)):
                flag=False
            if len(ReactionInputs(r,getSign(r)))==0 and m in ReactionProducts(r,getSign(r)):
                flag=False
        if flag:
#            print(m.id,"is an neither an input, nor an output")
            LMetOthers.append(m)
    LMet = LMetInputs+LMetOthers+LMetOutputs
    return LMet,LMetInputs,LMetOthers,LMetOutputs

def getMatD(model,V,atom):
    # get the list of metabolites
    LMet, LIn, LOthers, LOut = getMetabolitLists(model)
    MatD=np.zeros((len(LMet),len(LMet)))
    i=0
    for mi in LMet:
        k=0
        for mk in LMet:
            MatD[k,i]=value_of_D(mk,mi,V,"C")
            k+=1
        i+=1
    return matD

def PrintRepartition(model,V,atom="C",aff=True,precision=1e-10):
    ReactInputTable.clear()
    ReactProductTable.clear()
    EnrichModel(model)
    # get the list of metabolites
    res=dict()
    print("... Creating the metabolite lists")
    LMet, LIn, LOthers, LOut = getMetabolitLists(model)
    print("... Creating matrix D")
    MatD=np.zeros((len(LMet),len(LMet)))
    i=0
    for mi in LMet:
        k=0
        for mk in LMet:
            MatD[k,i]=value_of_D(mk,mi,V,atom)
            k+=1
        i+=1
    print("... Inverting matrix D")
    invmatD=np.linalg.inv(MatD)
    for i in range(len(LMet)):
        ch = "X_"+LMet[i].id+"*("+str(GetComposition(LMet[i].formula,atom))+")"+" = "
        res[LMet[i].id]=dict()
        for k in range(len(LIn)):
            if (invmatD[i,k] > precision and (GetComposition(LMet[k].formula,atom))>0.01):
                ch += " + "+str(invmatD[i,k])+" * X_"+LMet[k].id+"("+str(GetComposition(LMet[k].formula,atom))+")"
                res[LMet[i].id][LMet[k].id]=invmatD[i,k]
        if aff:
            print(ch)
    return res

isProducedBy=dict()
isConsumedBy=dict()

def EnrichModel(model):
    isProducedBy.clear()
    isConsumedBy.clear()
    for m in model.metabolites:
        isProducedBy[m.id]=list()
        isConsumedBy[m.id]=list()
        for r in m.reactions:
            if m in ReactionProducts(r,getSign(r)):
                isProducedBy[m.id].append(r)
            if m in ReactionInputs(r,getSign(r)):
                isConsumedBy[m.id].append(r)

def metRate(m,V):
    res=0
    for r in isProducedBy[m.id]:
        res+=r.get_coefficient(m)*V.get(r.id)
    return res

def metRate_inv(m,V):
    res=0
    for r in m.reactions:
        if m not in ReactionProducts(r,getSign(r)):
            res+=abs(r.get_coefficient(m))*V.get(r.id)
    return res

def FixModelFluxes(model,V):
    for r in model.reactions:
        value = V.get(r.id)
        r.lower_bound = value
        r.upper_bound = value



In [178]:
# pour compléter les modèles de BIGG au cas où il n'y aurait pas le champs "formula"

f=open("NA_Offline/chem_prop.tsv","r")

MNXDict=dict()

for l in f:
    t=l.split("\t")
    if (len(t)>2):
        MNXDict[t[0]]=t[2]
    
f.close()

f=open("NA_Offline/bigg_models_metabolites.txt","r")

MetDict=dict()

for l in f:
    t=l.split("\t")
    address=t[4]
    pos=address.find("http://identifiers.org/metanetx.chemical")
    address=address[pos:].replace("http://identifiers.org/metanetx.chemical/","")
    pos=address.find(";")
    if pos>0:
        address=address[:(pos)]
    MetDict[t[0]]=[address, MNXDict.get(address)]
    
f.close()





def Model_Missing_Formula(model):
    for m in model.metabolites:
        if m.formula == None:
            if (MetDict.get(m.id)[1] != None):
                m.formula = MetDict.get(m.id)[1]
            else:
                m.formula="C1"

# Test on a tiny example

## Model definition

In [179]:
from cobra import Model, Reaction, Metabolite
model = Model('example_model')

# set of reactions
reactions = dict()
for i in range(1,8):
    reactions["R"+str(i)] = Reaction("R"+str(i))
    reactions["R"+str(i)].name = "R"+str(i)
    reactions["R"+str(i)].subsystem = 'Body'
    reactions["R"+str(i)].lower_bound = 0.  # This is the default
    reactions["R"+str(i)].upper_bound = 1000.  # This is the default
#    reactions["R"+str(i)].objective_coefficient = 0. # this is the default
    print ("Creating R"+str(i))
# set of metabolites
A = Metabolite('A', name='A', compartment='c', formula = 'C1')
B = Metabolite('B', name='B', compartment='c', formula = 'C3')
C = Metabolite('C', name='C', compartment='c', formula = 'C5')
D = Metabolite('D', name='D', compartment='c', formula = 'C4')
E = Metabolite('E', name='E', compartment='c', formula = 'C2')

reactions["R1"].add_metabolites({A:-2, B:-1, C: 1})
reactions["R2"].add_metabolites({B:-1, C:-1, D: 1, E: 2})
reactions["R3"].add_metabolites({A:-4, D: 1})
reactions["R4"].add_metabolites({A:1})
reactions["R5"].add_metabolites({B:1})
reactions["R6"].add_metabolites({D: -1})
reactions["R7"].add_metabolites({E: -1})


reactions["R4"].lower_bound = 1
reactions["R4"].upper_bound = 6
#reactions["R4"].objective_coefficient = 1
reactions["R7"].lower_bound = 1
reactions["R7"].upper_bound = 4
#reactions["R7"].objective_coefficient = 1

for i in range(1,8):
    model.add_reaction(reactions["R"+str(i)])
    model.reactions.get_by_id("R"+str(i)).objective_coefficient = 0

model.objective = model.reactions.R4.flux_expression + model.reactions.R7.flux_expression


Creating R1
Creating R2
Creating R3
Creating R4
Creating R5
Creating R6
Creating R7


In [180]:
# Need to have the "real" lb and ub. Performing a FVA (with a f_to_opt=0.5) allows to get them
RescaleModel(model,0.5)

# Compute one vector
fba = model.optimize()
V=fba.x_dict
#FixModelFluxes(model,V)

PrintRepartition(model,V)

... Creating the metabolite lists
... Creating matrix D
... Inverting matrix D
X_A*(1) =  + 1.0 * X_A(1)
X_B*(3) =  + 1.0 * X_B(3)
X_C*(5) =  + 0.666666666667 * X_A(1) + 0.5 * X_B(3)
X_D*(4) =  + 0.666666666667 * X_A(1) + 0.5 * X_B(3)
X_E*(2) =  + 0.333333333333 * X_A(1) + 0.5 * X_B(3)


{'A': {'A': 1.0},
 'B': {'B': 1.0},
 'C': {'A': 0.66666666666666663, 'B': 0.5},
 'D': {'A': 0.66666666666666663, 'B': 0.5},
 'E': {'A': 0.33333333333333331, 'B': 0.5}}

# Test of a middle size example (e_coli_core from BIGG)

In [181]:
import time

globalstart = time.time()

start = time.time()
print("Reading the model")
model=cobra.io.read_sbml_model('NA_Offline/e_coli_core.xml')
end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Scaling the model")
RescaleModel(model,0.5)
Model_Missing_Formula(model)

end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Performing a FBA")
fba = model.optimize()
end = time.time()
print("       finishing in",end - start," s")
V=fba.x_dict

start = time.time()
print("Pick a sample flux vector")
VD=cobra.flux_analysis.sample(model,1).to_dict()
V=dict()
for r in VD:
    V[r]=VD[r].get(0)
end = time.time()
print("       finishing in",end - start," s")

    
start = time.time()
print("AIO Computation")
res=PrintRepartition(model,V)
end = time.time()
print("       finishing in",end - start," s")

globalend = time.time()
print("Global execution time = ",globalend - globalstart," s")


Reading the model
       finishing in 0.1217658519744873  s
Scaling the model
       finishing in 0.04843282699584961  s
Performing a FBA
       finishing in 0.0017139911651611328  s
Pick a sample flux vector
       finishing in 0.15105414390563965  s
AIO Computation
... Creating the metabolite lists
... Creating matrix D
... Inverting matrix D
X_glc__D_e*(6) =  + 1.0 * X_glc__D_e(6)
X_nh4_e*(0) = 
X_o2_e*(0) = 
X_pi_e*(0) = 
X_13dpg_c*(3) =  + 0.0672528412635 * X_glc__D_e(6)
X_2pg_c*(3) =  + 0.0456346669238 * X_glc__D_e(6)
X_3pg_c*(3) = 
X_6pgc_c*(6) =  + 0.234650260561 * X_glc__D_e(6)
X_6pgl_c*(6) =  + 0.234650260561 * X_glc__D_e(6)
X_ac_c*(2) = 
X_acald_c*(2) = 
X_accoa_c*(23) =  + 0.385157658251 * X_glc__D_e(6)
X_acon_C_c*(6) =  + 0.0508105497356 * X_glc__D_e(6)
X_actp_c*(2) =  + 0.000431662205185 * X_glc__D_e(6)
X_adp_c*(10) =  + 0.642081246606 * X_glc__D_e(6)
X_akg_c*(5) =  + 0.0465038030739 * X_glc__D_e(6)
X_amp_c*(10) =  + 0.103310964963 * X_glc__D_e(6)
X_atp_c*(10) =  + 0.3032

# Test of a large size example (Pseudomonas Putida from BIGG)

In [182]:
import time

globalstart = time.time()

start = time.time()
print("Reading the model")
model=cobra.io.read_sbml_model('NA_Offline/iJN746.xml')
end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Scaling the model")
RescaleModel(model,0.5)
Model_Missing_Formula(model)

end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Performing a FBA")
fba = model.optimize()
end = time.time()
print("       finishing in",end - start," s")
V=fba.x_dict

start = time.time()
print("Pick a sample flux vector")
VD=cobra.flux_analysis.sample(model,1).to_dict()
V=dict()
for r in VD:
    V[r]=VD[r].get(0)
end = time.time()
print("       finishing in",end - start," s")

    
start = time.time()
print("AIO Computation")
PrintRepartition(model,V)
end = time.time()
print("       finishing in",end - start," s")

globalend = time.time()
print("Global execution time = ",globalend - globalstart," s")


Reading the model
       finishing in 0.8422932624816895  s
Scaling the model
       finishing in 2.341830015182495  s
Performing a FBA
       finishing in 0.014899015426635742  s
Pick a sample flux vector
       finishing in 5.630018949508667  s
AIO Computation
... Creating the metabolite lists
... Creating matrix D
... Inverting matrix D
X_pqq_c*(14) =  + 1.0 * X_pqq_c(14)
X_dna5mtc_c*(0) = 
X_mclPHAg_c*(0) = 
X_glc__D_e*(6) =  + 1.0 * X_glc__D_e(6)
X_nh4_e*(0) = 
X_o2_e*(0) = 
X_pi_e*(0) = 
X_so4_e*(0) = 
X_co2_e*(1) = 
X_nadp_c*(21) = 
X_nadph_c*(21) =  + 1.23267035949 * X_pqq_c(14) + 0.118138403448 * X_glc__D_e(6)
X_Lpipecol_c*(6) =  + 2.25908322209e-07 * X_pqq_c(14) + 2.16509209506e-08 * X_glc__D_e(6)
X_1pipdn2c_c*(6) = 
X_h_c*(0) = 
X_atp_c*(10) =  + 0.27842864341 * X_pqq_c(14) + 0.118449595516 * X_glc__D_e(6)
X_2dhglcn_c*(6) = 
X_6p2dhglcn_c*(6) =  + 3.67296008152e-06 * X_pqq_c(14) + 1.56255703679e-06 * X_glc__D_e(6)
X_adp_c*(10) =  + 12.8744528758 * X_pqq_c(14) + 5.64258381046

# Test of a very large size example (Chlamydomonas reinhardtii from BIGG)

In [183]:
import time

globalstart = time.time()

start = time.time()
print("Reading the model")
model=cobra.io.read_sbml_model('NA_Offline/iRC1080.xml')
end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Scaling the model")
RescaleModel(model,0.5)
Model_Missing_Formula(model)

end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Performing a FBA")
fba = model.optimize()
end = time.time()
print("       finishing in",end - start," s")
V=fba.x_dict

start = time.time()
print("Pick a sample flux vector")
VD=cobra.flux_analysis.sample(model,1).to_dict()
V=dict()
for r in VD:
    V[r]=VD[r].get(0)
end = time.time()
print("       finishing in",end - start," s")

    
start = time.time()
print("AIO Computation")
PrintRepartition(model,V)
end = time.time()
print("       finishing in",end - start," s")

globalend = time.time()
print("Global execution time = ",globalend - globalstart," s")


Reading the model
       finishing in 1.7608318328857422  s
Scaling the model
       finishing in 11.39441704750061  s
Performing a FBA
       finishing in 0.10392904281616211  s
Pick a sample flux vector
       finishing in 30.509296894073486  s
AIO Computation
... Creating the metabolite lists
... Creating matrix D
... Inverting matrix D
X_mg2_e*(0) = 
X_na1_e*(0) = 
X_nh4_e*(0) = 
X_o2_e*(0) = 
X_photonVis_e*(0) = 
X_pi_e*(0) = 
X_so4_e*(0) = 
X_starch300_h*(0) = 
X_10fthf_c*(20) = 
X_10fthf_h*(20) = 
X_10fthf_m*(20) = 
X_10fthf_x*(20) = 
X_10fthfglu__L_c*(25) = 
X_10fthfglu__L_m*(25) = 
X_12dgr160_h*(35) = 
X_12dgr16018111Z_c*(37) = 
X_12dgr1601819Z_c*(37) = 
X_12dgr1601819Z_h*(37) = 
X_12dgr1801819Z_c*(39) = 
X_12dgr1801819Z_h*(39) = 
X_12dgr18111Z160_h*(0) = 
X_12dgr18111Z18111Z_c*(39) = 
X_12dgr18111Z1819Z_c*(39) = 
X_12dgr18111Z1819Z_h*(39) = 
X_12dgr1819Z160_h*(0) = 
X_12dgr1819Z1619Z_h*(0) = 
X_12dgr1819Z18111Z_c*(39) = 
X_12dgr1819Z18111Z_h*(39) = 
X_12dgr1819Z1819Z_c*(39) =

X_od2coa_c*(39) = 
X_omppp9_h*(35) = 
X_orn_c*(5) = 
X_orn_e*(5) = 
X_orn_h*(5) = 
X_orn_m*(5) = 
X_orot_c*(5) = 
X_orot_m*(5) = 
X_orot5p_c*(10) = 
X_oxa_m*(2) = 
X_pa_c*(18) = 
X_pa160_h*(35) = 
X_pa16018111Z_c*(37) = 
X_pa1601819Z_c*(37) = 
X_pa1601819Z_h*(37) = 
X_pa1801819Z_c*(39) = 
X_pa1801819Z_h*(39) = 
X_pa1801829Z12Z_c*(39) = 
X_pa1801835Z9Z12Z_c*(39) = 
X_pa1801845Z9Z12Z15Z_c*(39) = 
X_pa18111Z160_c*(37) = 
X_pa18111Z160_h*(37) = 
X_pa18111Z18111Z_c*(39) = 
X_pa18111Z1819Z_c*(39) = 
X_pa18111Z1819Z_h*(39) = 
X_pa18111Z1829Z12Z_c*(39) = 
X_pa18111Z1835Z9Z12Z_c*(39) = 
X_pa18111Z1845Z9Z12Z15Z_c*(39) = 
X_pa1819Z160_c*(37) = 
X_pa1819Z160_h*(37) = 
X_pa1819Z1619Z_h*(0) = 
X_pa1819Z18111Z_c*(39) = 
X_pa1819Z18111Z_h*(39) = 
X_pa1819Z1819Z_c*(39) = 
X_pa1819Z1819Z_h*(39) = 
X_pa1819Z1829Z12Z_c*(39) = 
X_pa1819Z1835Z9Z12Z_c*(39) = 
X_pa1819Z1845Z9Z12Z15Z_c*(39) = 
X_pa1829Z12Z1835Z9Z12Z_c*(39) = 
X_pacoa_c*(39) = 
X_pail18111Z160_c*(43) = 
X_pail18111Z160_e*(43) = 
X_pail1819Z160_

# Test on Synechococcus elongatus PCC 7942

In [186]:
import time

globalstart = time.time()

start = time.time()
print("Reading the model")
model=cobra.io.read_sbml_model('NA_Offline/iJB785.xml')
end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Scaling the model")
RescaleModel(model,0.5)
Model_Missing_Formula(model)

end = time.time()
print("       finishing in",end - start," s")

start = time.time()
print("Performing a FBA")
fba = model.optimize()
end = time.time()
print("       finishing in",end - start," s")
V=fba.x_dict

start = time.time()
print("Pick a sample flux vector")
VD=cobra.flux_analysis.sample(model,1).to_dict()
V=dict()
for r in VD:
    V[r]=VD[r].get(0)
end = time.time()
print("       finishing in",end - start," s")

    
start = time.time()
print("AIO Computation")
res=PrintRepartition(model,V)
end = time.time()
print("       finishing in",end - start," s")

globalend = time.time()
print("Global execution time = ",globalend - globalstart," s")


Reading the model
       finishing in 0.6645472049713135  s
Scaling the model
       finishing in 1.0451128482818604  s
Performing a FBA
       finishing in 0.015257835388183594  s
Pick a sample flux vector
       finishing in 3.9713032245635986  s
AIO Computation
... Creating the metabolite lists
... Creating matrix D
... Inverting matrix D
X_amylose_c*(0) = 
X_so4_e*(0) = 
X_no3_e*(0) = 
X_pi_e*(0) = 
X_photon410_e*(0) = 
X_photon430_e*(0) = 
X_photon450_e*(0) = 
X_photon470_e*(0) = 
X_photon490_e*(0) = 
X_photon510_e*(0) = 
X_photon530_e*(0) = 
X_photon550_e*(0) = 
X_photon570_e*(0) = 
X_photon590_e*(0) = 
X_photon610_e*(0) = 
X_photon630_e*(0) = 
X_photon650_e*(0) = 
X_photon670_e*(0) = 
X_photon690_e*(0) = 
X_leu__L_e*(6) =  + 1.0 * X_leu__L_e(6)
X_gam6p_c*(6) =  + 0.0684136467447 * X_leu__L_e(6)
X_cgly_c*(5) =  + 0.0512234056556 * X_leu__L_e(6)
X_achms_c*(6) = 
X_pcox_u*(0) = 
X_octe9ACP_c*(0) = 
X_fdp_c*(6) =  + 0.0039110410625 * X_leu__L_e(6)
X_agm_c*(5) = 
X_5caiz_c*(9) =  + 2

X_uacmam_c*(17) = 
X_udcpp_c*(55) =  + 0.21296936713 * X_leu__L_e(6)
X_unaga_c*(63) = 
X_kdolipid4cy_c*(84) = 
X_unagam_c*(71) = 
X_icolipacy_c*(1) = 
X_icolipacy_p*(1) = 
X_oanticy_c*(1) = 
X_oanticy_p*(1) = 
X_colipacy_p*(1) = 
X_colipacy_e*(1) = 
X_uaagmda_c*(95) =  + 0.367852845238 * X_leu__L_e(6)
X_murein5p5p_p*(80) =  + 0.150278613323 * X_leu__L_e(6)
X_murein5p5p5p_p*(120) =  + 0.0138203855953 * X_leu__L_e(6)
X_ala__D_p*(3) =  + 0.0116148380419 * X_leu__L_e(6)
X_murein5px4px4p_p*(114) =  + 0.0131095100849 * X_leu__L_e(6)
X_murein5px4p_p*(77) =  + 0.125083639068 * X_leu__L_e(6)
X_alaala_p*(6) = 
X_murein5px3p_p*(74) = 
X_murein4px4p_p*(74) =  + 0.0250677719147 * X_leu__L_e(6)
X_murein4px4px4p_p*(111) =  + 0.0127645229774 * X_leu__L_e(6)
X_murein5p4p_p*(77) =  + 0.11854572072 * X_leu__L_e(6)
X_murein4p4p_p*(74) =  + 0.138994828192 * X_leu__L_e(6)
X_murein4p3p_p*(71) = 
X_murein5p3p_p*(74) = 
X_murein3px4p_p*(71) = 
X_murein4px4p4p_p*(111) =  + 0.0127645229774 * X_leu__L_e(6)
X_mure

In [169]:
# misc. Une tentative d'écriture de convertisseur Modèle Cobra vers modèle nutrition analyzer (.mdl)
# Attention, nutrition analyzer ne traite que des réactions irréversibles et dont le flux est positif. 
# Le convertisseur gère les réactions qui ne sont pas écrites dans le modèle cobra dans le bon sens mais
# il faut que les lower et upper bounds soient de même signe.

def toMDL(model,atom="C"):
    # reactions
    i=1
    ch=""
    for r in model.reactions:
        ch+=str(i)+";"+r.id
        for m in model.metabolites:
            if m in r.metabolites:
                if (r.lower_bound>=0):
                    ch+=";"+str(r.get_coefficient(m))
                else:
                    ch+=";"+str(-r.get_coefficient(m))
            else:
                ch+=";0"
        ch+=";0\n"
        i+=1
    # repartition carbones
    ch+=str(i)+";Carbones"
    for m in model.metabolites:
        ch+=";"+str(GetComposition(m.formula,atom))
    ch+=";1\n"
    i+=1
    
    ch="BEGIN Matrice\n"+ch+"END\n"
    
    # Nom colonnes
    
    chcolname="ID\nR_Name\n"
    for m in model.metabolites:
        chcolname+=m.name+"\n"
    chcolname+="Repart. Carbones\n"
    chcolname="BEGIN NomsColonnes\n"+chcolname+"END\n"

    
    # Type colonnes
    chcoltype="2\n1\n"
    for m in model.metabolites:
        chcoltype+="3\n"
    chcoltype+="9\n"
    chcoltype="BEGIN TypesColonnes\n"+chcoltype+"END\n"
    
    # reactions utilisées
    chReactut="0\n"
    for r in model.reactions:
        chReactut+="on\n"
    chReactut+="0\n"
    chReactut="BEGIN EquationsUtilisees\n"+chReactut+"END\n"


    # Contraintes et objectif
    chReactres=""
    objectif="Maximize "
    i=1;
    for r in model.reactions:
        if (r.lower_bound>=0):
            chReactres+="R_"+str(i)+">"+str(r.lower_bound)+"\n"
            chReactres+="R_"+str(i)+"<"+str(r.upper_bound)+"\n"
        else:
            chReactres+="R_"+str(i)+">"+str(-r.upper_bound)+"\n"
            chReactres+="R_"+str(i)+"<"+str(-r.lower_bound)+"\n"            
        if (not r.objective_coefficient == 0):
            if (r.lower_bound>=0):
                sign=1
            else:
                sign=-1
            if r.objective_coefficient*sign == 1:
                objectif+= "+"+str(sign)+"* R_"+str(i);
            else:
                objectif+= " + "+str(sign*r.objective_coefficient)+" * R_"+str(i);
        i+=1

                
    chReactres="BEGIN contraintes\n"+chReactres+objectif+"\nEND\n"
    
    
    return ch+chReactut+chcolname+chcoltype+chReactres

In [185]:
# exemple d'utilisation du convertisseur avec e coli core model

model= cobra.io.read_sbml_model("NA_Offline/e_coli_core.xml")

# on calcule un vecteur "optimal" pour avoir le sens des réactions
fba = model.optimize()
# on modifie les lower/upper bounds en conséquence
for r in model.reactions:
    if (fba.x_dict.get(r.id)>0):
        r.lower_bound=0
    else: 
        r.upper_bound=0

# export du modèle qui peut être utilisé dans nutrition analyzer
fd=open("SortieExpl.mdl","w")
fd.write(toMDL(model))
fd.close()